In [4]:
import math
import os
import janitor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from dotenv import load_dotenv
from google.cloud import bigquery
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, max_error
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, RobustScaler, StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LSTM, LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
load_dotenv()
import os

# my functions
from mymodule import create_csv_file
from mymodule import remove_pattern
from mymodule import trim_to_three_cd10
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/amermujkanovic/code/Amerigo23/Maximon/gcp.json'
from data_and_lists import drg_codes, diesaes_columns, disease_icd10_dict, disease_icd10_names , state_codes, state_column_names, sorted_discharge_status


In [6]:
google_credentials = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
PROJECT = "focus-hulling-416322"
DATASET = "2016"
TABLE = "ccaei163_narrow"

# Ensure the project, dataset, and table names are correctly formatted
# using backticks to avoid syntax errors
query = f"""
    SELECT *
    FROM `{PROJECT}.{DATASET}.{TABLE}`
"""

client = bigquery.Client(project=PROJECT)
query_job = client.query(query)
result = query_job.result()
data = result.to_dataframe()
data = data.applymap(remove_pattern)


/var/folders/fh/fc20820s23gdqf7bkj31z2rw0000gn/T/ipykernel_11263/2237358714.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(remove_pattern)


In [12]:
disease_icd10_dict

{'Acute Myocardial Infarction (Heart Attack)': 'I21',
 'Angina Pectoris': 'I20',
 'Aortic Aneurysm': 'I71',
 'Peripheral Artery Disease': 'I73',
 'Thrombophlebitis': 'I80',
 'Varicose Veins': 'I83',
 'Heart Failure': 'I50',
 'Atrial Fibrillation': 'I48',
 'Cardiomyopathy': 'I42',
 'Mitral Valve Disorders': 'I34',
 'Rheumatic Heart Disease': 'I01',
 'Hypertensive Heart Disease': 'I11',
 'Venous Insufficiency': 'I87.2',
 "Raynaud's Syndrome": 'I73.0',
 'Acute Bronchitis': 'J20',
 'Chronic Bronchitis': 'J42',
 'Pneumonia': 'J18',
 'Pulmonary Edema': 'J81',
 'Pulmonary Embolism': 'I26',
 'Pleurisy': 'R09.1',
 'Asthma': 'J45',
 'Emphysema': 'J43',
 'Sleep Apnea': 'G47.3',
 'Lung Cancer': 'C34',
 'Interstitial Lung Disease': 'J84',
 'Sarcoidosis': 'D86',
 'Cystic Fibrosis': 'E84',
 'Pulmonary Hypertension': 'I27',
 'Tuberculosis (TB)': 'A15',
 'Silicosis': 'J62',
 'Acute Upper Respiratory Infections': 'J06',
 'Influenza': 'J09-J11',
 'Tonsillitis': 'J03',
 'Pharyngitis': 'J02',
 'Laryngitis'

In [10]:
df = data.clean_names()
trim_to_three_cd10(df, disease_icd10_names)

for disease_name, icd_code in disease_icd10_dict.items():
    df[disease_name] = df.apply(lambda row: any(row[col] == icd_code for col in disease_icd10_names), axis=1).astype(int)

# # Drop the disease code columns after processing
# df = df.drop(diesaes_columns, axis=1)

# # Process region codes
# for code, state_name in state_codes.items():
#     df[state_name] = (df['state'] == code).astype(int)

# # Drop the region_code column after processing
# df = df.drop('state', axis=1)

# df['drg'] = df['drg'].map(drg_codes)
# df['egeoloc'] = df['egeoloc'].map(state_column_names )
df


Column 'Acute Myocardial Infarction (Heart Attack)' not found in the dataframe
Column 'Angina Pectoris' not found in the dataframe
Column 'Aortic Aneurysm' not found in the dataframe
Column 'Peripheral Artery Disease' not found in the dataframe
Column 'Thrombophlebitis' not found in the dataframe
Column 'Varicose Veins' not found in the dataframe
Column 'Heart Failure' not found in the dataframe
Column 'Atrial Fibrillation' not found in the dataframe
Column 'Cardiomyopathy' not found in the dataframe
Column 'Mitral Valve Disorders' not found in the dataframe
Column 'Rheumatic Heart Disease' not found in the dataframe
Column 'Hypertensive Heart Disease' not found in the dataframe
Column 'Venous Insufficiency' not found in the dataframe
Column 'Raynaud's Syndrome' not found in the dataframe
Column 'Acute Bronchitis' not found in the dataframe
Column 'Chronic Bronchitis' not found in the dataframe
Column 'Pneumonia' not found in the dataframe
Column 'Pulmonary Edema' not found in the data

KeyError: 'Acute Myocardial Infarction (Heart Attack)'

In [25]:
columns_to_groupby = ['efamid',
                     'enrolid',
                     'eidflag',
                     'sex',
                     'region',
                     'drg',
                     'caseid',
                     'rx',
                     'plantyp',
                     'hlthplan']

columns_to_sum = (diesaes_columns_names +
               state_column_names +
               ['days',
               'physnet',
               'physpay',
               'totcob',
               'totcoins',
               'totcopay',
               'totded',
               'totnet',
               'totpay'
               ]
)
columns_to_mean = ['age']

# aggregation_dictionary
aggregation_function = {col: 'sum' for col in columns_to_sum}
aggregation_function.update({col: 'mean' for col in columns_to_mean})

grouped_df = df.groupby(columns_to_groupby).agg(aggregation_function)
grouped_df


Acute Myocardial Infarction (Heart Attack)  \
efamid    enrolid     eidflag sex region drg                                                caseid  rx plantyp hlthplan                                               
682       68202       1       2   2      ALCOHOL                                            14418   1  6.0     0                                                  0   
1476      147601      1       1   2      FRACTURE                                           14419   1  6.0     0                                                  0   
3674      367401      1       2   3      POISONING AND TOXIC EFFECTS OF DRUGS WITH MCC      14421   1  6.0     0                                                  0   
10938     1093802     1       2   3      KIDNEY AND URINARY TRACT INFECTIONS WITHOUT MCC    14430   1  6.0     0                                                  0   
11100     1110002     1       1   3      CELLULITIS WITHOUT MCC                             14432   1  6.0     0                                                  0   
...                                                                                                                                                             ...   
340437840 34043784001 1       1   3      MAJOR HIP AND KNEE JOINT REPLACEMENT OR REATTAC... 1622508 1  5.0     1                                                  0   
                                         PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITH NON... 1622507 1  5.0     1                                                  1   
340437957 34043795701 1       1   3      REVISION OF HIP OR KNEE REPLACEMENT WITHOUT CC/MCC 1622520 1  5.0     1                                                  0   
340438010 34043801001 1       2   3      MAJOR HIP AND KNEE JOINT REPLACEMENT OR REATTAC... 1622523 1  5.0     1                                                  0   
340438090 34043809005 1       2   3      NEONATE WITH OTHER SIGNIFICANT PROBLEMS            1622534 1  5.0     1                                                  0   

                                                                                                                         Angina Pectoris  \
efamid    enrolid     eidflag sex region drg                                                caseid  rx plantyp hlthplan                    
682       68202       1       2   2      ALCOHOL                                            14418   1  6.0     0                       0   
1476      147601      1       1   2      FRACTURE                                           14419   1  6.0     0                       0   
3674      367401      1       2   3      POISONING AND TOXIC EFFECTS OF DRUGS WITH MCC      14421   1  6.0     0                       0   
10938     1093802     1       2   3      KIDNEY AND URINARY TRACT INFECTIONS WITHOUT MCC    14430   1  6.0     0                       0   
11100     1110002     1       1   3      CELLULITIS WITHOUT MCC                             14432   1  6.0     0                       0   
...                                                                                                                                  ...   
340437840 34043784001 1       1   3      MAJOR HIP AND KNEE JOINT REPLACEMENT OR REATTAC... 1622508 1  5.0     1                       0   
                                         PERCUTANEOUS CARDIOVASCULAR PROCEDURES WITH NON... 1622507 1  5.0     1                       0   
340437957 34043795701 1       1   3      REVISION OF HIP OR KNEE REPLACEMENT WITHOUT CC/MCC 1622520 1  5.0     1                       0   
340438010 34043801001 1       2   3      MAJOR HIP AND KNEE JOINT REPLACEMENT OR REATTAC... 1622523 1  5.0     1                       0   
340438090 34043809005 1       2   3      NEONATE WITH OTHER SIGNIFICANT PROBLEMS            1622534 1  5.0     1                       0   

                                                                                                                         Aortic 

In [26]:
duplicates = df.duplicated('enrolid',  keep=False)
count_duplicates = duplicates.sum()
count_duplicates

168259

In [27]:
duplicate_counts = df[df.duplicated('enrolid', keep=False)]['enrolid'].value_counts()
print(duplicate_counts)

enrolid
32702962003    29
2859418406     25
1059540402     23
3592415004     23
28468331003    23
               ..
3557164001      2
2675731802      2
1260407102      2
1269669801      2
1038904001      2
Name: count, Length: 61475, dtype: int64


/var/folders/fh/fc20820s23gdqf7bkj31z2rw0000gn/T/ipykernel_55577/2077537071.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[disease_name] = df.apply(lambda row: any (row[col] == icd_code for col in ['pdx', 'dx1','dx2','dx3','dx4','dx5','dx6','dx7','dx8','dx9','dx10','dx11','dx12','dx13','dx14','dx15']), axis=1).astype(int)
/var/folders/fh/fc20820s23gdqf7bkj31z2rw0000gn/T/ipykernel_55577/2077537071.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[disease_name] = df.apply(lambda row: any (row[col] == icd_code for col 

,efamid,enrolid,eidflag,physid,caseid,age,sex,region,state,days,drg,disdate,hospnet,hosppay,mhsacovg,pdx,dx1,dx2,dx3,dx4,dx5,dx6,dx7,dx8,dx9,dx10,dx11,dx12,dx13,dx14,dx15,dstatus,physnet,physpay,plantyp,hlthplan,pproc,eeclass,eestatu,egeoloc,emprel,rx,totcob,totcoins,totcopay,totded,totnet,totpay,Acute Myocardial Infarction (Heart Attack),Angina Pectoris,Aortic Aneurysm,Peripheral Artery Disease,Thrombophlebitis,Varicose Veins,Heart Failure,Atrial Fibrillation,Cardiomyopathy,Mitral Valve Disorders,Rheumatic Heart Disease,Hypertensive Heart Disease,Venous Insufficiency,Raynaud's Syndrome,Acute Bronchitis,Chronic Bronchitis,Pneumonia,Pulmonary Edema,Pulmonary Embolism,Pleurisy,Asthma,Emphysema,Sleep Apnea,Lung Cancer,Interstitial Lung Disease,Sarcoidosis,Cystic Fibrosis,Pulmonary Hypertension,Tuberculosis (TB),Silicosis,Acute Upper Respiratory Infections,Influenza,Tonsillitis,Pharyngitis,Laryngitis,Sinusitis,Otitis Media,Meniere's Disease,Hearing Loss,Vertigo,Tinnitus,Conjunctivitis,Cataract,Glaucoma,Diabetic Retinopathy,Macular Degeneration,Refractive Errors,Strabismus,Corneal Ulcer,Dry Eye Syndrome,Retinal Detachment,Uveitis,Keratitis,Blepharitis,Stye (Hordeolum),Chalazion,Acanthamoeba Keratitis,Trachoma,Acute Appendicitis,Cholecystitis,Pancreatitis,Hepatitis A,Hepatitis B,Hepatitis C,Alcoholic Liver Disease,Non-Alcoholic Fatty Liver Disease,Cirrhosis,Gastric Ulcer,Duodenal Ulcer,Gastroenteritis,Diverticulitis,Irritable Bowel Syndrome,Inflammatory Bowel Disease,Celiac Disease,Lactose Intolerance,Intestinal Obstruction,Hemorrhoids,Anal Fissure,Rectal Prolapse,Esophageal Varices,Gastroesophageal Reflux Disease (GERD),Peptic Ulcer Disease,Chronic Kidney Disease,Nephrolithiasis (Kidney Stones),Acute Kidney Injury,Glomerulonephritis,Urinary Tract Infection,Bladder Cancer,Prostate Cancer,Benign Prostatic Hyperplasia,Erectile Dysfunction,Urinary Incontinence,Interstitial Cystitis,Polycystic Kidney Disease,Acute Cystitis,Pyelonephritis,Urethritis,Male Infertility,Female Infertility,Polycystic Ovary Syndrome (PCOS)
0,39581991,3958199102,1,29842521.0,1346649,61,2,5,49,5,389,2016-02-05,15329.51,15329.51,1,K56,K56,G89,N13,R05,R10,K59,C20,K31,R06,R11,C79,R64,R68,Non,Non,01,689.45,689.45,2.0,0,4,5,1,01,2,1,0.0,0.0,0.0,0.00,17388.85,17388.85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,39601661,3960166101,1,118615807.0,1348901,60,2,5,49,15,7,2016-10-27,206820.26,206820.26,1,J84,J84,I34,R07,R07,J84,J84,Z94,R73,Z76,M25,I36,J18,J96,I10,I51,06,22367.44,22795.91,6.0,0,4,1,9,01,1,1,0.0,0.0,0.0,428.47,229718.40,230146.87,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,297791054,29779105402,1,NaN,245966,51,2,5,06,3,377,2016-09-10,34863.04,24168.92,1,K92,K92,D50,D62,E83,E87,E87,F10,F32,G93,I49,Z45,D50,K21,K44,K52,07,1248.60,1248.60,6.0,1,None,9,9,01,2,1,0.0,0.0,600.0,0.00,36733.48,26039.36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,332969309,33296930902,1,462658955.0,1445011,54,2,5,33,5,92,2016-09-21,9518.40,11898.00,1,R26,R26,D64,F10,F12,F17,G89,M48,M54,R25,M62,M79,M54,R26,Non,Non,01,280.13,280.13,6.0,1,None,9,1,01,2,1,0.0,2379.6,0.0,0.00,9887.48,12267.08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,333683659,33368365901,1,658800401.0,1479311,54,1,5,64,1,392,2016-08-02,10108.40,10108.40,1,K31,K31,D68,E87,E87,E87,I50,K20,K25,M06,N28,R10,R91,R11,R07,R19,01,435.95,435.95,6.0,1,None,9,1,01,1,1,0.0,0.0,0.0,0.00,10987.88,10987.88,0,0,0,0,0,

In [20]:
 # Example with 50 columns
df_1 = pd.DataFrame(df)

# Set Pandas display options to show all columns
pd.set_option('display.max_columns', None)

# Display the DataFrame
df_1

,efamid,enrolid,eidflag,physid,caseid,age,sex,region,days,drg,disdate,hospnet,hosppay,mhsacovg,dstatus,physnet,physpay,plantyp,hlthplan,pproc,eeclass,eestatu,egeoloc,emprel,rx,totcob,totcoins,totcopay,totded,totnet,totpay,Acute Myocardial Infarction (Heart Attack),Angina Pectoris,Aortic Aneurysm,Peripheral Artery Disease,Thrombophlebitis,Varicose Veins,Heart Failure,Atrial Fibrillation,Cardiomyopathy,Mitral Valve Disorders,Rheumatic Heart Disease,Hypertensive Heart Disease,Venous Insufficiency,Raynaud's Syndrome,Acute Bronchitis,Chronic Bronchitis,Pneumonia,Pulmonary Edema,Pulmonary Embolism,Pleurisy,Asthma,Emphysema,Sleep Apnea,Lung Cancer,Interstitial Lung Disease,Sarcoidosis,Cystic Fibrosis,Pulmonary Hypertension,Tuberculosis (TB),Silicosis,Acute Upper Respiratory Infections,Influenza,Tonsillitis,Pharyngitis,Laryngitis,Sinusitis,Otitis Media,Meniere's Disease,Hearing Loss,Vertigo,Tinnitus,Conjunctivitis,Cataract,Glaucoma,Diabetic Retinopathy,Macular Degeneration,Refractive Errors,Strabismus,Corneal Ulcer,Dry Eye Syndrome,Retinal Detachment,Uveitis,Keratitis,Blepharitis,Stye (Hordeolum),Chalazion,Acanthamoeba Keratitis,Trachoma,Acute Appendicitis,Cholecystitis,Pancreatitis,Hepatitis A,Hepatitis B,Hepatitis C,Alcoholic Liver Disease,Non-Alcoholic Fatty Liver Disease,Cirrhosis,Gastric Ulcer,Duodenal Ulcer,Gastroenteritis,Diverticulitis,Irritable Bowel Syndrome,Inflammatory Bowel Disease,Celiac Disease,Lactose Intolerance,Intestinal Obstruction,Hemorrhoids,Anal Fissure,Rectal Prolapse,Esophageal Varices,Gastroesophageal Reflux Disease (GERD),Peptic Ulcer Disease,Chronic Kidney Disease,Nephrolithiasis (Kidney Stones),Acute Kidney Injury,Glomerulonephritis,Urinary Tract Infection,Bladder Cancer,Prostate Cancer,Benign Prostatic Hyperplasia,Erectile Dysfunction,Urinary Incontinence,Interstitial Cystitis,Polycystic Kidney Disease,Acute Cystitis,Pyelonephritis,Urethritis,Male Infertility,Female Infertility,Polycystic Ovary Syndrome (PCOS),"Nation, unknown region","Northeast Region, unknown division","New England Division, unknown state",Connecticut,Maine,Massachusetts,New Hampshire,Rhode Island,Vermont,"Middle Atlantic Division, unknown state",New Jersey,New York,Pennsylvania,"North Central Region, unknown division","East North Central Division, unknown state",Illinois,Indiana,Michigan,Ohio,Wisconsin,"West North Central Division, unknown state",Iowa,Kansas,Minnesota,Missouri,Nebraska,North Dakota,South Dakota,"South Region, unknown division","South Atlantic Division, unknown state","Washington, DC",Delaware,Florida,Georgia,Maryland,North Carolina,South Carolina,Virginia,West Virginia,"East South Central Division, unknown state",Alabama,Kentucky,Mississippi,Tennessee,"West South Central Division, unknown state",Arkansas,Louisiana,Oklahoma,Texas,"West Region, unknown division","Mountain Division, unknown state",Arizona,Colorado,Idaho,Montana,Nevada,New Mexico,Utah,Wyoming,"Pacific Division, unknown state",Alaska,California,Hawaii,Oregon,Washington,Puerto Rico
0,39581991,3958199102,1,29842521.0,1346649,61,2,5,5,G.I. OBSTRUCTION WITH CC,2016-02-05,15329.51,15329.51,1,01,689.45,689.45,2.0,0,4,5,1,01,2,1,0.0,0.0,0.0,0.00,17388.85,17388.85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,39601661,3960166101,1,118615807.0,1348901,60,2,5,15,LUNG TRANSPLANT,2016-10-27,206820.26,206820.26,1,06,22367.44,22795.91,6.0,0,4,1,9,01,1,1,0.0,0.0,0.0,428.47,229718.40,230146.87,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,